In [11]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

# Modeling Exercises

In [12]:
# pull in data
df = pd.read_csv('saas.csv')
df.head()

,Month_Invoiced,Customer_Id,Invoice_Id,Subscription_Type,Amount
0,2014-01-31,1000000,5000000,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0


In [13]:
# check summary statistics for amount column
df.Amount.describe()

count    790458.000000
mean          2.996174
std           4.751901
min           0.000000
25%           0.000000
50%           0.000000
75%           5.000000
max          20.000000
Name: Amount, dtype: float64

In [14]:
# check value counts for amount column
df.Amount.value_counts()

0.0     496410
5.0     178212
10.0     70706
15.0     26474
20.0     18656
Name: Amount, dtype: int64

In [15]:
# check info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790458 entries, 0 to 790457
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Month_Invoiced     790458 non-null  object 
 1   Customer_Id        790458 non-null  int64  
 2   Invoice_Id         790458 non-null  int64  
 3   Subscription_Type  790458 non-null  float64
 4   Amount             790458 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 30.2+ MB


In [17]:
# drop unnecessary columns
df.drop(columns=['Customer_Id', 'Invoice_Id', 'Subscription_Type'], inplace=True)
df.head(2)

,Month_Invoiced,Amount
0,2014-01-31,0.0
1,2014-01-31,10.0


In [18]:
# rename columns
df = df.rename(columns={'Month_Invoiced': 'bill_date', 'Amount':'amount'})
df.head(2)

,bill_date,amount
0,2014-01-31,0.0
1,2014-01-31,10.0


In [19]:
# change Month_Invoiced column to DataTime 
df.bill_date = pd.to_datetime(df.bill_date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790458 entries, 0 to 790457
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   bill_date  790458 non-null  datetime64[ns]
 1   amount     790458 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 12.1 MB


In [21]:
# set bill_date column as index and sort values
df = df.set_index('bill_date').sort_index()
df.head(2)

,amount
bill_date,
2014-01-31,0.0
2014-01-31,10.0


In [22]:
# check range of data
df.index.min(), df.index.max()

(Timestamp('2014-01-31 00:00:00'), Timestamp('2017-12-31 00:00:00'))

In [32]:
# sum up the amount of each month's sales
df = df.resample('M').amount.sum()
df.shape

(48,)

## Exercise 1

Split data (train/validate/test) and resample by any period, except daily, and aggregate using the sum.